In [0]:
DECLARE OR REPLACE VARIABLE catalog_use = 'main';
DECLARE OR REPLACE VARIABLE schema_use = 'synthea';

In [0]:
SET VARIABLE catalog_use = :catalog_use;
SET VARIABLE schema_use = :schema_use; 

In [0]:
USE IDENTIFIER(catalog_use || '.' || schema_use);
SELECT current_catalog(), current_schema();

In [0]:
SHOW FUNCTIONS;

In [0]:
SELECT
  databricks_rest_get(
    endpoint => '/2.1/unity-catalog'
    ,resource => 'tables'
    ,path_parameters => NULL
    ,query_parameters => array('catalog_name=mgiglia', 'schema_name=synthea')
    ,body => NULL
  )

In [0]:
CREATE OR REPLACE FUNCTION databricks_rest_unity_catalog(
  conn STRING COMMENT 'The name of the Unity Catalog HTTP Connection for the Databricks REST API.'
  ,method STRING DEFAULT 'GET' COMMENT 'The HTTP Method to use for the request to the Unity Catalog API.'
  ,resource STRING COMMENT "The Unity Catalog API asset or service to perform actions against such as 'Tables', 'Catalogs', 'Artifact Allow Lists', etc."
  ,path_parameters ARRAY<STRING> COMMENT "An array of path parameters to append to the Unity Catalog API resource url if applicable."
  ,query_parameters ARRAY<STRING> COMMENT "An array of query parameters to apply to the endpoint URL."
  ,body STRING COMMENT "The JSON body of the request to send to the Unity Catalog API as a string."
)
RETURNS VARIANT
COMMENT 'A function to perform actions against the Unity Catalog Databricks REST API.'
LANGUAGE SQL
RETURN (
  SELECT
    parse_json(
      http_request(
        conn => conn
        ,method => method
        ,path => CASE when path_parameters IS NULL and query_parameters IS NULL THEN concat('/2.1/unity-catalog/', resource)
                      when path_parameters IS NULL and query_parameters IS NOT NULL THEN concat('/2.1/unity-catalog/', resource, '?', array_join(query_parameters, '&')) 
                      when path_parameters IS NOT NULL and query_parameters IS NULL THEN concat('/2.1/unity-catalog/', resource, '/', array_join(path_parameters, '/'))
                      ELSE concat('/2.1/unity-catalog/', resource, '/', array_join(path_parameters, '/'), '?', array_join(query_parameters, '&')) END
        ,json => CASE when body IS NULL THEN NULL else to_json(body) END
      ).text
    ) as resp_variant
)